# Database Creator

Need to make/finish the functions to make the domain and subdomain lists

In [1]:
from security_domains import SECURITY_DOMAINS

In [2]:
envs = {
    'dev': {
        'env_prefix': 'tbdd',
        'service_account_suffix': 'd',
        'sensitive_suffix': 'sensitive',
        'env_suffix': 'dev',
    },
    'qa': {
        'env_prefix': 'tbdq',
        'service_account_suffix': 'q',
        'sensitive_suffix': 'sensitive',
        'env_suffix': 'qa',
    },
    'prod': {
        'env_prefix': 'tbdp',
        'service_account_suffix': 'p',
        'sensitive_suffix': 'sensitive',
        'env_suffix': 'prod',
    },
}


In [178]:
def make_security_groups_from_domain_subdomain(domain, subdomain):
    pass


def add_domain(env=None, info=None, template=False):
    """Use template=True to output an example dictionary"""
    pass


def add_subdomain(env=None, domain=None, subdomains=None, template=False):
    """Use template=True to output an example dictionary"""
    sensitive = info.get('sensitive')
    for subdomain in subdomains:
        for sg in subdomain.get('security_groups'):
            add_security_group(
                env=env, domain=domain, subdomain=subdomain, sensitive=sensitive
            )
    pass

In [ ]:
# ADD_SECURITY_GROUP IS WORKING
def add_security_group(env, domain, subdomain=None, sensitive=None):
    env_prefix = env.get('env_prefix')
    sensitive_suffix = env.get('sensitive_suffix')
    group_parts = [env_prefix, domain]
    if subdomain is not None:
        group_parts.append(subdomain)
    if sensitive is not None:
        if sensitive == 'replace':
            # add senstivie_suffix to end
            group_parts.append(sensitive_suffix)
        elif sensitive == 'add_extra':
            # add extra security group with sensitive_suffix
            group_parts_sen = group_parts.copy()
            group_parts_sen.append(sensitive_suffix)
            security_group_sen = '-'.join(group_parts_sen)
            add_group_sen_sql = f'''add group "{security_group_sen}";'''
        else:
            raise ValueError(
                f'sensitive must be either `add_extra` or `replace`, got `{sensitive}`'
            )
    security_group = '-'.join(group_parts)
    add_group_sql = f'''add group "{security_group}";'''
    if sensitive == 'add_extra':
        return [add_group_sql, add_group_sen_sql]
    else:
        return [add_group_sql]


In [185]:
add_security_group(
    env=envs.get('dev'), domain='customer', subdomain='master', sensitive='add_extra'
)


['add group "tbdd-customer-master";',
 'add group "tbdd-customer-master-sensitive";']

In [40]:
# PRINT_DATABASE_DOMAINS IS WORKING
def print_database_domains(database, indent_chars=10):
    """prints the database domains, subdomains, and security groups"""

    def print_indent(item, level, type):
        indent_string = ' ' * level * indent_chars
        sep = '-'
        if type == 'security_domain':
            print_string = item.upper() + ' -- Security Domain'
            print(print_string)
            print(sep * len(print_string))
        elif type == 'domain':
            print_string = indent_string + item.upper() + ' -- Data Domain'
            print(print_string)
            print(
                ' ' * len(indent_string) + sep * (len(print_string) - len(indent_string))
            )
        elif type == 'subdomain':
            print_string = indent_string + item.upper() + ' -- Subdomain'
            print(print_string)
            print(
                ' ' * len(indent_string) + sep * (len(print_string) - len(indent_string))
            )
        elif type == 'security_group':
            print(indent_string + item)

    def display_and_interpret_domain(domains, level, type):
        for name, domain in domains.items():
            print()
            print_indent(name, level, type=type)
            interpret(domain, level)

    def interpret(thing, level=-1):
        security_domains = thing.get('security_domains')
        domains = thing.get('domains')
        subdomains = thing.get('subdomains')
        security_groups = thing.get('security_groups')
        if security_groups is not None:
            for security_group in security_groups:
                print_indent(f"'{security_group}'", level, type='security_group')
        if subdomains is not None:
            level += 1
            display_and_interpret_domain(
                domains=subdomains, level=level, type='subdomain'
            )
        if domains is not None:
            level += 1
            display_and_interpret_domain(
                domains=domains, level=level, type='domain'
            )
        if security_domains is not None:
            level += 1
            display_and_interpret_domain(
                domains=security_domains, level=level, type='security_domain'
            )

    return interpret(database)


In [ ]:
print_database_domains(SECURITY_DOMAINS, indent_chars=8)

In [13]:
# GET_DOMAIN_SECURITY_GROUPS IS WORKING
def get_domain_security_groups(database):
    """Get a list of data domain security groups"""
    security_group_list = []
    def reinterpret(domains):
        for name, domain in domains.items():
            interpret(domain)

    def interpret(thing):
        security_domains = thing.get('security_domains')
        domains = thing.get('domains')
        subdomains = thing.get('subdomains')
        security_groups = thing.get('security_groups')
        if security_groups is not None:
            for security_group in security_groups:
                if security_group != '':
                    security_group_list.append(security_group)
        if subdomains is not None:
            reinterpret(subdomains)
        if domains is not None:
            reinterpret(domains)
        if security_domains is not None:
            reinterpret(security_domains)
    
    interpret(database)
    return security_group_list


In [14]:
get_domain_security_groups(SECURITY_DOMAINS)

['customer-master',
 'customer-vehicle-purchase-writeup',
 'call-center-toy',
 'call-center-toy-sen',
 'customer-demographics',
 'customer-communication-management-sen',
 'customer-dealer-relationship',
 'customer-parts-ecommerce',
 'customer-predictive-score',
 'customer-predictive-score-toyota',
 'customer-predictive-score-lexus',
 'customer-alias',
 'customer-marketing',
 'party-vendor-partner',
 'non-customer-sensitive',
 'dealer-master',
 'party-dealer-management',
 'party-dealer-management-sensitive',
 'dealer-personnel',
 'dealer-call-center',
 'party-region-master-sensitive',
 'dealership-staff',
 'vehicle-master-data',
 'vehicle-master-data-sensitive',
 'vehicle-master-data-lexus-sensitive',
 'vehicle-master-data-toyota-sensitive',
 'vehicle-sales',
 'sales-actuals-sen',
 'sales-forecast',
 'sales-forecast-factor',
 'sales-forecast-sen',
 'vin-dist-sales-sen',
 'sales-incentives-forecast-sen',
 'vehicle-sales-new-car-aggregated-actuals',
 'vehicle-sales-new-car-vin-based-actua